## Import Packages and Initialize SparkSession

In [1]:
from IPython.display import Image # for Importing coverpage Image
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum, avg, count, desc

# Initialize SparkSession
spark = SparkSession.builder.appName("Data Processing and Analysis").getOrCreate()


# Warning Suppression
import warnings
warnings.filterwarnings("ignore")

print("Libraries imported successfully!")

Libraries imported successfully!


## Create DataFrames from Various Data Sources
### Load Sales CSV

In [2]:
# Load sales data from CSV
sales_df = spark.read.csv("sales_data.csv", header=True, inferSchema=True)

sales_df.show(10)

+--------------+----------+-----------+------+
|Transaction_ID|      Date|Customer_ID|Amount|
+--------------+----------+-----------+------+
|             1|2024-01-01|         37|178.94|
|             2|2024-01-02|          1| 236.3|
|             3|2024-01-03|         67|213.11|
|             4|2024-01-04|         24| 16.49|
|             5|2024-01-05|         27|155.01|
|             6|2024-01-06|         40| 235.0|
|             7|2024-01-07|         63|211.44|
|             8|2024-01-08|         61|363.75|
|             9|2024-01-09|         36|198.55|
|            10|2024-01-10|         98|249.79|
+--------------+----------+-----------+------+
only showing top 10 rows



#### Load Customer JSON

In [3]:
# Load customer data from JSON
customers_df = spark.read.json("customers_data.json")

# Show the first 10 rows
customers_df.show(10)

+---+-----------+------+--------+-----------+
|Age|Customer_ID|Gender|Location|       Name|
+---+-----------+------+--------+-----------+
| 69|          1|Female|  City_B| Customer_1|
| 21|          2|  Male|  City_B| Customer_2|
| 39|          3|Female|  City_C| Customer_3|
| 43|          4|  Male|  City_A| Customer_4|
| 66|          5|Female|  City_B| Customer_5|
| 60|          6|  Male|  City_C| Customer_6|
| 53|          7|  Male|  City_A| Customer_7|
| 30|          8|Female|  City_A| Customer_8|
| 69|          9|Female|  City_B| Customer_9|
| 49|         10|  Male|  City_B|Customer_10|
+---+-----------+------+--------+-----------+
only showing top 10 rows



#### Load Product Parque

In [4]:
# Load product data from Parquet
products_df = spark.read.parquet("products_data.parquet")

# Show the first 10 rows
products_df.show(10)

+----------+------------+-----------+-----+------+
|Product_ID|Product_Name|   Category|Stock| Price|
+----------+------------+-----------+-----+------+
|         1|   Product_1|    Grocery|   41|199.41|
|         2|   Product_2|  Furniture|   83|145.43|
|         3|   Product_3|  Furniture|    5| 61.01|
|         4|   Product_4|Electronics|   32|176.55|
|         5|   Product_5|    Grocery|   86|  14.6|
|         6|   Product_6|   Clothing|   70| 50.06|
|         7|   Product_7|   Clothing|   35|179.99|
|         8|   Product_8|    Grocery|   22| 54.96|
|         9|   Product_9|   Clothing|   65| 48.03|
|        10|  Product_10|  Furniture|   84| 31.93|
+----------+------------+-----------+-----+------+
only showing top 10 rows



## Data Cleaning
#### Handle Missing or Null Values

In [5]:
# Handle missing values in sales
sales_df = sales_df.na.fill({"Amount": 0})

customers_df = customers_df.na.drop()
products_df = products_df.na.drop()


In [6]:
# Check schemas after cleaning
sales_df.printSchema()
customers_df.printSchema()
products_df.printSchema()

root
 |-- Transaction_ID: integer (nullable = true)
 |-- Date: date (nullable = true)
 |-- Customer_ID: integer (nullable = true)
 |-- Amount: double (nullable = false)

root
 |-- Age: long (nullable = true)
 |-- Customer_ID: long (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Name: string (nullable = true)

root
 |-- Product_ID: integer (nullable = true)
 |-- Product_Name: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Stock: integer (nullable = true)
 |-- Price: double (nullable = true)



## Data Transformation
#### Filter Data

In [7]:
# Filter sales with Amount > 100
filtered_sales = sales_df.filter(col("Amount") > 100)
filtered_sales.show(10)

+--------------+----------+-----------+------+
|Transaction_ID|      Date|Customer_ID|Amount|
+--------------+----------+-----------+------+
|             1|2024-01-01|         37|178.94|
|             2|2024-01-02|          1| 236.3|
|             3|2024-01-03|         67|213.11|
|             5|2024-01-05|         27|155.01|
|             6|2024-01-06|         40| 235.0|
|             7|2024-01-07|         63|211.44|
|             8|2024-01-08|         61|363.75|
|             9|2024-01-09|         36|198.55|
|            10|2024-01-10|         98|249.79|
|            11|2024-01-11|         93|220.51|
+--------------+----------+-----------+------+
only showing top 10 rows



#### Group and Aggregate

In [8]:
# Total sales by customer
total_sales_per_customer = sales_df.groupBy("Customer_ID").agg(sum("Amount").alias("Total_Sales"))
total_sales_per_customer.show(10)

+-----------+------------------+
|Customer_ID|       Total_Sales|
+-----------+------------------+
|         31|            109.68|
|         85|2227.5800000000004|
|         65|           1637.09|
|         53|           1612.51|
|         78|           1693.21|
|         34|            1417.0|
|         81|           1693.54|
|         28|           1466.08|
|         76|           1188.26|
|         27|           2258.75|
+-----------+------------------+
only showing top 10 rows



#### Sorting

In [9]:
# Top 5 customers by total sales
top_customers = total_sales_per_customer.orderBy(desc("Total_Sales")).limit(5)
top_customers.show()

+-----------+------------------+
|Customer_ID|       Total_Sales|
+-----------+------------------+
|         68|3132.9599999999996|
|         69|            3092.2|
|         41|2903.0399999999995|
|          6|           2832.02|
|         57|2477.4500000000003|
+-----------+------------------+



### Register DataFrames as Temporary Views

In [10]:
# Register DataFrames as temporary views
sales_df.createOrReplaceTempView("sales")
customers_df.createOrReplaceTempView("customers")
products_df.createOrReplaceTempView("products")

## Perform Basic SQL Operations
#### Find Total Sales Per Customer

In [11]:
query = """
SELECT s.Customer_ID, c.Name, SUM(s.Amount) AS Total_Sales
FROM sales s
JOIN customers c ON s.Customer_ID = c.Customer_ID
GROUP BY s.Customer_ID, c.Name
ORDER BY Total_Sales DESC
"""
total_sales = spark.sql(query)
total_sales.show(5)

+-----------+-----------+------------------+
|Customer_ID|       Name|       Total_Sales|
+-----------+-----------+------------------+
|         68|Customer_68|3132.9599999999996|
|         69|Customer_69|            3092.2|
|         41|Customer_41|2903.0399999999995|
|          6| Customer_6|           2832.02|
|         57|Customer_57|2477.4500000000003|
+-----------+-----------+------------------+
only showing top 5 rows



#### Join and Filter Data

In [12]:
query = """
SELECT s.Transaction_ID, c.Name, p.Product_Name, s.Amount, p.Stock
FROM sales s
JOIN customers c ON s.Customer_ID = c.Customer_ID
JOIN products p ON s.Customer_ID = p.Product_ID
WHERE p.Stock < 10
"""
low_stock_sales = spark.sql(query)
low_stock_sales.show(5)

+--------------+-----------+------------+------+-----+
|Transaction_ID|       Name|Product_Name|Amount|Stock|
+--------------+-----------+------------+------+-----+
|            17|Customer_57|  Product_57|275.27|    2|
|            20|Customer_53|  Product_53|368.88|    7|
|            22|Customer_69|  Product_69| 69.53|    6|
|            24|Customer_53|  Product_53|295.25|    7|
|            27|Customer_57|  Product_57|162.39|    2|
+--------------+-----------+------------+------+-----+
only showing top 5 rows



## Advanced SQL Operations
#### Total Sales by Product Category

In [13]:
query = """
SELECT p.Category, SUM(s.Amount) AS Total_Sales
FROM sales s
JOIN products p ON s.Customer_ID = p.Product_ID
GROUP BY p.Category
ORDER BY Total_Sales DESC
"""
sales_by_category = spark.sql(query)
sales_by_category.show(4)

+-----------+------------------+
|   Category|       Total_Sales|
+-----------+------------------+
|    Grocery| 46969.13000000002|
|Electronics|32562.999999999996|
|  Furniture|          28376.51|
|   Clothing|25499.309999999994|
+-----------+------------------+



#### Average Transaction Amount by Customer Location

In [14]:
query = """
SELECT c.Location, AVG(s.Amount) AS Avg_Transaction
FROM sales s
JOIN customers c ON s.Customer_ID = c.Customer_ID
GROUP BY c.Location
ORDER BY Avg_Transaction DESC
"""
avg_transaction_by_location = spark.sql(query)
avg_transaction_by_location.show(3)

+--------+-----------------+
|Location|  Avg_Transaction|
+--------+-----------------+
|  City_A|278.1170700636942|
|  City_C|274.2276502732242|
|  City_B|247.2494375000002|
+--------+-----------------+



#### Report Insights
##### Top-Selling Products: Products from high-demand categories (e.g., electronics) should have higher stock levels to prevent shortages.
##### High-Value Customers: The top 5 customers contribute significantly to overall sales, warranting targeted loyalty programs.
##### Customer Location Trends: Customers from specific regions spend more, indicating potential for location-based marketing campaigns.
##### Low Stock Products: Products with less than 10 units in stock need immediate replenishment to avoid lost sales opportunities.